In [1]:
import tensorflow as tf

In [2]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
x_train,x_test = x_train.astype('float32'), x_test.astype('float32')

x_train,x_test = x_train.reshape([-1,784]), x_test.reshape([-1, 784])
x_train, x_test = x_train/255. , x_test/ 255.


In [4]:
# 학습에 필요한 설정값들을 정의합니다.
learning_rate_RMSProp = 0.02
learning_rate_GradientDescent = 0.5
num_epochs = 100         # 반복횟수
batch_size = 256
display_step = 1         # 몇 Step마다 log를 출력할지 결정합니다.
input_size = 784         # MNIST 데이터 input (이미지 크기: 28*28)
hidden1_size = 128       # 첫번째 히든레이어의 노드 개수 
hidden2_size = 64        # 두번째 히든레이어의 노드 개수 


In [5]:
train_data = tf.data.Dataset.from_tensor_slices(( x_train, y_train ))
train_data = train_data.shuffle(60000).batch(batch_size)

In [6]:
def random_normal_intializer_with_stddev_1():
  return tf.keras.initializers.RandomNormal(mean=0.0, stddev=1.0, seed=None)

In [11]:
# tf.keras.Model을 이용해서 Autoencoder 모델을 정의합니다.
class AutoEncoder(tf.keras.Model):
  def __init__(self):
    super(AutoEncoder, self).__init__()
    # 인코딩(Encoding) - 784 -> 128 -> 64
    self.hidden_layer_1 = tf.keras.layers.Dense(hidden1_size,
                                                activation='sigmoid',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())
    self.hidden_layer_2 = tf.keras.layers.Dense(hidden2_size,
                                                activation='sigmoid',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())
    # 디코딩(Decoding) 64 -> 128 -> 784
    self.hidden_layer_3 = tf.keras.layers.Dense(hidden1_size,
                                                activation='sigmoid',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())
    self.output_layer = tf.keras.layers.Dense(input_size,
                                                activation='sigmoid',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())

  def call(self, x):
    H1_output = self.hidden_layer_1(x)
    H2_output = self.hidden_layer_2(H1_output)
    H3_output = self.hidden_layer_3(H2_output)
    X_reconstructed = self.output_layer(H3_output)

    return X_reconstructed, H2_output

In [14]:
#####대체 왜 invalid syntax가 나는 거야?????
### ???????????????


class AutoEncoder (tf.keras.Model):
    def _init_(self):
      super( AutoEncoder, self)._init_()

       # 인코딩(Encoding) - 784 -> 128 -> 64
      self.hidden_layer_1 = tf.keras.layers.Dense (hidden1_size, 
                                                  activation= 'sigmoid',
                                                  kernel_initializer = random_normal_intializer_with_stddev_1(),
                                                  bias_initializer = random_normal_intializer_with_stddev_1())
    
      self.hidden_layer_2 = tf.keras.layers.Dense (hidden2_size, 
                                                  activation= 'sigmoid',
                                                  kernel_initializer = random_normal_intializer_with_stddev_1(),

       # 디코딩(Decoding) 64 -> 128 -> 784 
      self.hidden_layer_3 = tf.keras.layers.Dense (hidden1_size, 
                                                  activation= 'sigmoid',
                                                  kernel_initializer = random_normal_intializer_with_stddev_1(),
                                                  bias_initializer = random_normal_intializer_with_stddev_1())
                                              
      self.output_layer = tf.keras.layers.Dense(input_size, 
                                                  activation= 'sigmoid',
                                                  kernel_initializer = random_normal_intializer_with_stddev_1(),
                                                  bias_initializer = random_normal_intializer_with_stddev_1())
    def call(self,x):
      H1_output = self.hidden_layer_1(x)
      H2_output = self.hidden2_layer_2(H1_output)
      H3_output = self.hidden3_layer_2(H2_output)
      X_reconstructed = self.output_layer(H3_output)

      return  X_reconstructed, H2_output

SyntaxError: ignored

In [15]:
class SoftmaxClassifier (tf.keras.Model):
    def _init_(self):
      super(SoftmaxClassifier, self)._init_()
      self.softmax_layer = tf.keras.layers.Dense (10, 
                                                  activation= 'softmax',
                                                  kernel_initializer = 'zeros',
                                                  bias_initializer = 'zeros')
    def call(self,x):
      y_pred = self.softmax_layer (x)

      return y_pred

def pretraining_mse_loss(y_pred, y_true):
  return tf.reduce_mean(tf.pow(y_true - y_pred, 2))

def finetuning_cross_entropy_loss(y_pred_softmax, y):
  return tf.reduce_mean(-tf.reduce_mean(y * tf.math.log(y_pred_softmax), axis=[1]))  

pretraining_optimizer = tf.optimizers.RMSprop(learning_rate_RMSProp, epsilon= 1e-10)

def pretraining_train_step (autoencoder_model, x):
  y_true = x
  with tf.GradientTape() as tape:
    y_pred, _ = autoencoder_model(x)
    pretraining_loss = pretraining_mse_loss (y_pred, y_true)
  gradients = tape.gradient(pretraining_loss, autoencoder_model.trainable_variables)
  pretraining_optimizer.apply_gradients(zip(gradients, autoencoder_model.trainable_variables))



In [ ]:
# 10000 Step만큼 최적화를 수행합니다.
for i in range(10000):
  batch_x, batch_y = next(train_data_iter)

  # 100 Step마다 training 데이터셋에 대한 정확도와 loss를 출력합니다.
  if i % 100 == 0:
    train_accuracy = compute_accuracy(CNN_model(batch_x)[0], batch_y)
    print("반복(Epoch): %d, 트레이닝 데이터 정확도: %f" % (i, train_accuracy))

  train_step(CNN_model, batch_x, batch_y )

print(" 정확도: %f" % compute_accuracy(CNN_model(x_test)[0], y_test)) 

반복(Epoch): 0, 트레이닝 데이터 정확도: 0.060000
반복(Epoch): 100, 트레이닝 데이터 정확도: 0.760000
반복(Epoch): 200, 트레이닝 데이터 정확도: 0.920000
반복(Epoch): 300, 트레이닝 데이터 정확도: 0.880000
반복(Epoch): 400, 트레이닝 데이터 정확도: 0.940000
반복(Epoch): 500, 트레이닝 데이터 정확도: 0.980000
반복(Epoch): 600, 트레이닝 데이터 정확도: 0.980000
반복(Epoch): 700, 트레이닝 데이터 정확도: 0.980000
반복(Epoch): 800, 트레이닝 데이터 정확도: 0.980000
반복(Epoch): 900, 트레이닝 데이터 정확도: 1.000000
반복(Epoch): 1000, 트레이닝 데이터 정확도: 0.980000
반복(Epoch): 1100, 트레이닝 데이터 정확도: 0.980000
반복(Epoch): 1200, 트레이닝 데이터 정확도: 0.980000
반복(Epoch): 1300, 트레이닝 데이터 정확도: 1.000000
반복(Epoch): 1400, 트레이닝 데이터 정확도: 1.000000


KeyboardInterrupt: ignored